 Multiclass classification

In [2]:
import pandas as pd
cars = pd.read_csv("auto.csv")
print(cars.head())
unique_regions=cars["origin"].unique()
print(unique_regions)

    mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0  18.0          8         307.0       130.0  3504.0          12.0    70   
1  15.0          8         350.0       165.0  3693.0          11.5    70   
2  18.0          8         318.0       150.0  3436.0          11.0    70   
3  16.0          8         304.0       150.0  3433.0          12.0    70   
4  17.0          8         302.0       140.0  3449.0          10.5    70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  
[1 3 2]


Dummy variables

In previous classification missions, categorical variables have been represented in the dataset using integer values (like 0 and 1) for us already. In many cases, like with this dataset, you'll have to create numeric representation of categorical values yourself. For this dataset, categorical variables exist in three columns, cylinders, year, and origin. The cylinders and year columns must be converted to numeric values so we can use them to predict label origin. Even though the column year is a number, we’re going to treat them like categories. The year 71 is unlikely to relate to the year 70 in the same way those two numbers do numerically, but rather just as two different labels. In these instances, it is always safer to treat discrete values as categorical variables.

We must use dummy variables for columns containing categorical values. Whenever we have more than 2 categories, we need to create more columns to represent the categories. Since we have 5 different categories of cylinders, we could use 3, 4, 5, 6, and 8 to represent the different categories. We can split the column into separate binary columns:

    cyl_3 -- Does the car have 3 cylinders? 0 if False, 1 if True.
    cyl_4 -- Does the car have 4 cylinders? 0 if False, 1 if True.
    cyl_5 -- Does the car have 5 cylinders? 0 if False, 1 if True.
    cyl_6 -- Does the car have 6 cylinders? 0 if False, 1 if True.
    cyl_8 -- Does the car have 8 cylinders? 0 if False, 1 if True.

We can use the pandas.get_dummies() function to return a Dataframe containing binary columns from the values in the cylinders column. In addition, if we set the prefix parameter to cyl, Pandas will pre-pend the column names to match the style we'd like:

dummy_df = pd.get_dummies(cars["cylinders"], prefix="cyl")

We then use the pandas.concat() function to add the columns from this Dataframe back to cars:


In [3]:
dummy_cylinders = pd.get_dummies(cars["cylinders"], prefix="cyl")
cars = pd.concat([cars, dummy_cylinders], axis=1)
dummy_years=pd.get_dummies(cars["year"],prefix="year")
cars=pd.concat([cars,dummy_years],axis=1)
cars=cars.drop("cylinders",axis=1)
cars=cars.drop("year",axis=1)
cars.head(5)

,mpg,displacement,horsepower,weight,acceleration,origin,cyl_3,cyl_4,cyl_5,cyl_6,...,year_73,year_74,year_75,year_76,year_77,year_78,year_79,year_80,year_81,year_82
0,18.0,307.0,130.0,3504.0,12.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,350.0,165.0,3693.0,11.5,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,18.0,318.0,150.0,3436.0,11.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.0,304.0,150.0,3433.0,12.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17.0,302.0,140.0,3449.0,10.5,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Multiclass classification

In previous missions, we explored binary classification, where there were only 2 possible categories, or classes. When we have 3 or more categories, we call the problem a multiclass classification problem. There are a few different methods of doing multiclass classification and in this mission, we'll focus on the one-versus-all method.

The one-versus-all method is a technique where we choose a single category as the Positive case and group the rest of the categories as the False case. We're essentially splitting the problem into multiple binary classification problems. For each observation, the model will then output the probability of belonging to each category.

To start let's split our data into a training and test set. We've randomized the cars Dataframe for you already to start things off and assigned the shuffled Dataframe to shuffled_cars.


In [5]:
import numpy as np
shuffled_rows = np.random.permutation(cars.index)
shuffled_cars = cars.iloc[shuffled_rows]
perc_70=int(cars.shape[0]*0.7)
train=shuffled_cars.iloc[0:perc_70]
test=shuffled_cars.iloc[perc_70:]

Training a multiclass logistic regression model

In the one-vs-all approach, we're essentially converting an n-class (in our case n is 3) classification problem into n binary classification problems. For our case, we'll need to train 3 models:

    A model where all cars built in North America are considered Positive (1) and those built in Europe and Asia are considered Negative (0).
    A model where all cars built in Europe are considered Positive (1) and those built in North America and Asia are considered Negative (0).
    A model where all cars built in Asia are labeled Positive (1) and those built in North America and Europe are considered Negative (0).

Each of these models is a binary classification model that will return a probability between 0 and 1. When we apply this model on new data, a probability value will be returned from each model (3 total). For each observation, we choose the label corresponding to the model that predicted the highest probability.

We'll use the dummy variables we created from the cylinders and year columns to train 3 models using the LogisticRegression class from scikit-learn.


In [6]:
from sklearn.linear_model import LogisticRegression

unique_origins = cars["origin"].unique()
unique_origins.sort()

models = {}
features=[cols for cols in train.columns if (cols.startswith("cyl") or cols.startswith("year"))]
for origin in unique_origins:
       model=LogisticRegression()
       X_train=train[features]
       y_train=train["origin"]==origin
       model.fit(X_train,y_train)  
       models[origin]=model 

In [9]:
testing_probs = pd.DataFrame(columns=unique_origins)

for origin in unique_origins:
    # Select testing features.
    X_test = test[features]   
    # Compute probability of observation being in the origin.
    testing_probs[origin] = models[origin].predict_proba(X_test)[:,1]
print(testing_probs.head(5))    

          1         2         3
0  0.959759  0.031620  0.028345
1  0.219210  0.458958  0.337435
2  0.543053  0.177169  0.278065
3  0.959759  0.031620  0.028345
4  0.739878  0.122698  0.106255


Choose the origin

Now that we trained the models and computed the probabilities in each origin we can classify each observation. To classify each observation we want to select the origin with the highest probability of classification for that observation.

While each column in our dataframe testing_probs represents an origin we just need to choose the one with the largest probability. We can use the Dataframe method .idxmax() to return a Series where each value corresponds to the column or where the maximum value occurs for that observation. We need to make sure to set the axis paramater to 1 since we want to calculate the maximum value across columns. Since each column maps directly to an origin the resulting Series will be the classification from our model.

In [11]:
predicted_origins = testing_probs.idxmax(axis=1)
print(predicted_origins.head())

0    1
1    2
2    1
3    1
4    1
dtype: int64
